# 🛒 SQS Pizza Shop — Customer Notebook

You are a **customer** at the pizza shop. Your job is to:
1. **Place an order** by entering your name and choosing toppings
2. **Send the order** to the shared `pizza-orders` queue
3. **Wait for your pizza** by polling the `pizza-results` queue for your unique Correlation ID

You will see which cook made your pizza when the confirmation arrives!

---

## How This Works (Behind the Scenes)

When you submit an order, your notebook:
- Generates a **Correlation ID** (a unique UUID) — this is your ticket number
- Sends a JSON message to the **order queue** containing your name, toppings, and the Correlation ID
- Then polls the **response queue**, looking for a message tagged with YOUR Correlation ID

Meanwhile, a cook (another student in the class) is running the **Cook Notebook**. They pull your order from the order queue, "cook" it, and post a result back to the response queue with your Correlation ID attached. That's how your notebook knows which response belongs to you, even though dozens of students are all sharing the same queues.

---

## Setup: AWS Credentials

If you already have your Colab Secrets configured from Lab 1, just run this cell. Otherwise:
1. Click the **🔑 key icon** in the left sidebar
2. Add `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY`
3. Toggle **Notebook access** ON for each

In [ ]:
!pip install -q boto3

import boto3, os, json, time, uuid
from botocore.exceptions import ClientError

# --- Load credentials ---
try:
    from google.colab import userdata
    os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
    os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
    print('✅ Loaded credentials from Colab Secrets')
except (ImportError, KeyError):
    print('Not in Colab — using default AWS credential chain')

# --- Verify connection ---
sts = boto3.client('sts', region_name='us-east-1')
identity = sts.get_caller_identity()
print(f"Connected as: {identity['Arn']}")

REGION = 'us-east-1'
sqs = boto3.client('sqs', region_name=REGION)
print('SQS client ready.')

## Step 1: Connect to the Shared Queues

The queue URLs are pre-configured. Just run this cell to verify connectivity.

**What are these?** These are the addresses of the two SQS queues that the entire class shares. One is for sending orders (the ticket rack), the other is for receiving confirmations (the pickup counter). Your instructor already created them.

In [ ]:
#  Queue URLs (pre-configured by your instructor)
ORDER_QUEUE_URL = 'https://sqs.us-east-1.amazonaws.com/194722398367/pizza-orders'
RESPONSE_QUEUE_URL = 'https://sqs.us-east-1.amazonaws.com/194722398367/pizza-results'

# Quick sanity check — does the queue exist?
try:
    attrs = sqs.get_queue_attributes(QueueUrl=ORDER_QUEUE_URL, AttributeNames=['QueueArn'])
    print(f"✅ Order queue connected: {attrs['Attributes']['QueueArn']}")
    attrs = sqs.get_queue_attributes(QueueUrl=RESPONSE_QUEUE_URL, AttributeNames=['QueueArn'])
    print(f"✅ Response queue connected: {attrs['Attributes']['QueueArn']}")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Check with your instructor if the queues are set up.")

## Step 2: Place Your Order

Enter your name and choose your toppings. When you run this cell, it builds a **JSON message** — the same format that real APIs use to communicate.

The cell also generates a **Correlation ID** — a unique UUID that acts as your ticket number. You'll use this later to find your specific response in the shared response queue.

### Why a Correlation ID?
Imagine 20 students all sending orders to the same queue. When a cook posts a "pizza ready!" message back, how does YOUR notebook know it's YOUR pizza? The Correlation ID solves this — it's a unique string that travels with your order through the entire system, from order → cook → result → back to you.

In [ ]:
# --- YOUR ORDER ---
CUSTOMER_NAME = 'YOUR NAME HERE'  # <-- Put your real name!

# Pick your toppings (edit this list)
TOPPINGS = ['pepperoni', 'mushrooms', 'extra cheese']  # <-- Change these!

# --- Generate a unique ticket number ---
correlation_id = str(uuid.uuid4())

# --- Build the order message ---
order = {
    'correlation_id': correlation_id,
    'customer': CUSTOMER_NAME,
    'toppings': TOPPINGS,
    'ordered_at': time.strftime('%H:%M:%S')
}

print(f"Your order:")
print(f"  Customer:  {CUSTOMER_NAME}")
print(f"  Toppings:  {', '.join(TOPPINGS)}")
print(f"  Ticket #:  {correlation_id[:8]}...")
print(f"\nReady to send! Run the next cell to submit.")

## Step 3: Submit the Order

This cell calls `sqs.send_message()` — the exact same API call that a real e-commerce backend would use to enqueue a task.

Notice how fast this completes. The message is placed on the queue in **milliseconds**. The cook hasn't seen it yet, and that's the whole point — **temporal decoupling**. Your notebook doesn't wait for the pizza to be made. It just drops the ticket on the rack and moves on.

In [ ]:
# --- Send the order to the shared queue ---
response = sqs.send_message(
    QueueUrl=ORDER_QUEUE_URL,
    MessageBody=json.dumps(order)
)

print(f"✅ Order submitted!")
print(f"   SQS Message ID: {response['MessageId'][:12]}...")
print(f"   Your ticket #:  {correlation_id[:8]}...")
print(f"\n   Now waiting for a cook to pick it up...")
print(f"   Run the next cell to start waiting for your pizza.")

## Step 4: Wait for Your Pizza

This cell polls the **response queue** looking for a message that contains YOUR Correlation ID. It uses **Long Polling** (`WaitTimeSeconds=5`) — this means the API call will wait up to 5 seconds for a message to arrive before returning empty. This is more efficient than hitting the API every second.

### What's happening behind the scenes:
1. A cook (another student) pulls your order from the order queue
2. They see your name and toppings
3. They "cook" your pizza (their notebook simulates work)
4. They post a result to the response queue with YOUR Correlation ID
5. This cell detects that message and shows you the confirmation

### Why check the Correlation ID?
Because OTHER customers' results are also in the same response queue! Without checking the ID, you might accidentally pick up someone else's confirmation. This cell reads messages, checks if the Correlation ID matches yours, and if it doesn't, puts it back so the rightful customer can find it.

In [ ]:
# --- Poll for YOUR result ---
print(f"Waiting for pizza (ticket #{correlation_id[:8]}...)")
print(f"This will poll for up to 2 minutes.\n")

found = False
for attempt in range(24):  # 24 attempts × 5 second waits = 2 minutes max
    response = sqs.receive_message(
        QueueUrl=RESPONSE_QUEUE_URL,
        MaxNumberOfMessages=10,
        WaitTimeSeconds=5
    )
    
    messages = response.get('Messages', [])
    for msg in messages:
        result = json.loads(msg['Body'])
        
        if result.get('correlation_id') == correlation_id:
            # THIS IS OUR PIZZA!
            sqs.delete_message(QueueUrl=RESPONSE_QUEUE_URL, ReceiptHandle=msg['ReceiptHandle'])
            print(f"🍕 YOUR PIZZA IS READY!")
            print(f"   Cooked by: {result.get('cook', 'Unknown')}")
            print(f"   Status:    {result.get('status', 'READY')}")
            print(f"   Ticket #:  {correlation_id[:8]}...")
            found = True
            break
        else:
            # Not our pizza — put it back by NOT deleting it.
            # The visibility timeout will make it visible again.
            pass

    if found:
        break
    print(f"  Still waiting... (attempt {attempt + 1}/24)")

if not found:
    print(f"\n⏰ Timed out after 2 minutes. No cook has responded yet.")
    print(f"   This can happen if all cooks are busy or haven't started their notebook.")

## 🎉 Done!

Take a **screenshot** of your output above showing:
- Your order details (name, toppings)
- The confirmation with your cook's name

Upload the screenshot to the **Canvas assignment**.

---

### Want to send another order?
Go back to **Step 2**, change your toppings, and re-run Steps 2 → 3 → 4. Each order gets a new Correlation ID, so there's no collision with your previous order.